<h1><img width="30" src="../resources/gradio-icon.png"> Iris Gradio</h1>

### 📥 Importing

In [49]:
! pip install gradio

In [50]:
import gradio as gr

theme = gr.themes.Base(
    primary_hue="violet",
    secondary_hue="teal",
    neutral_hue="slate",
    font_mono=[gr.themes.GoogleFont('JetBrains Mono'), 'ui-monospace', 'Consolas', 'monospace'],
)

set_darkmode = """
function refresh() {
    const url = new URL(window.location);

    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

### 🖐️ 안녕, 세상아!

In [55]:
def greet(name):
    return f'환영합니다, {name}님!' if name else '👀'

In [56]:
demo = gr.Interface(fn=greet, inputs='text', outputs='text', theme=theme, js=set_darkmode)
demo.launch()

* Running on local URL:  http://127.0.0.1:7896

To create a public link, set `share=True` in `launch()`.


In [58]:
with gr.Blocks(theme=theme, js=set_darkmode) as demo:
    gr.Markdown('# 🖐️ Hello, World!')
    
    gr.Markdown('## 이름을 입력하고 버튼을 누르면...?')
    with gr.Row():
        input_textbox = gr.Textbox(label='📛 이름을 입력하세요.')
        output_textbox = gr.Textbox(label='🖐️ 어서오세요!', interactive=False)
    send_button = gr.Button('전송', variant='primary')
    send_button.click(fn=greet, inputs=input_textbox, outputs=output_textbox)
    
    gr.Markdown('## 이번엔 이름을 입력하기만 해도?')
    with gr.Row():
        input_textbox = gr.Textbox(label='📛 이름을 입력하세요.')
        output_textbox = gr.Textbox(label='🖐️ 어서오세요!', interactive=False)
    input_textbox.change(fn=greet, inputs=input_textbox, outputs=output_textbox)
    
demo.launch()

* Running on local URL:  http://127.0.0.1:7898

To create a public link, set `share=True` in `launch()`.
